In [1]:

'''
#--------------
#BOZZA PROGETTO
#--------------
#Machine Learning (apprendere dai mondiali passati)
#ranking fifa
#fifa 18 stats
#quote snai
#partite ultimo anno
#allenatore
#situazione economica
#twitter
#--------------
#DATI
#--------------
https://www.kaggle.com/thec03u5/fifa-18-demo-player-dataset#CompleteDataset.csv
https://www.kaggle.com/ahmedelnaggar/fifa-worldcup-2018-dataset/version/1#World%20Cup%202018%20Dataset.csv
https://raw.githubusercontent.com/kafagy/fifa-FUT-Data/master/FIFA18.csv
---------------
TODO
---------------
debug codice
inserire i dati
regolare i parametri
twitter
machine learning
'''
import csv
from difflib import SequenceMatcher

In [2]:
class Player():
    def __init__(self):
        self.stats = {"overall" : None, "pace" : None, "shooting" : None, "passing" : None, "dribbling" : None, "defending" : None, "phisichal" : None} #Various stats between 0 and 100   ->
        self.name = None  # Player's name
        self.team = None  # Player's team
        self.age = None  # Player's age
        self.nAccidents = None  # Number of the player's accidents in his career
        self.position = None  # Position covered by the player
        self.regular = None  # It defines if the player is a regular or not
        
    def to_string(self):
        return "name: " + str(self.name) + " overall: " + str(self.stats.get("overall"))

In [3]:
class Match():  
    def __init__(self, team1, team2):
        self.teams = [team1, team2]   

    def result_calculator(self, delta = 30):  # if a team win return this team else return None
        x=-0.5 # ranking moltiplicator
        y=1 # power moltiplicatpor
        z = 5 # Average pace moltiplicator
        winProbability=(self.teams[0].rank*x + self.teams[0].power*y + self.teams[0].averagePace*z)-(self.teams[1].rank*x + self.teams[1].power*y + self.teams[1].averagePace*z)
        if abs(winProbability)<delta:
            return None
        if winProbability>0:
            return self.teams[0]
        else:
            return self.teams[1]        

In [4]:
class WorldCup():
    def __init__(self, year, groups):
        self.year = year   # Year in which lived the competition
        self.groups = groups  # World cup initial groups
        self.knockout_phase = [[]for i in range(5)]
        self.knockout_phase[0] = range(16)
        
    def calculate_match_list(self):  # Calculate the match list for every group
        for group in self.groups:
            group.match_list()
    
    def pre_rounds(self): # Initalize the knockout phase
        
        for x in range (0,len(self.groups)):
            for i in range (0,len(self.groups[x].teams)-1):
                for j in range(i, len(self.groups[x].teams)):
                    if (self.groups[x].scores[j]>self.groups[x].scores[i]):
                        self.groups[x].teams[i], self.groups[x].teams[j] = self.groups[x].teams[j], self.groups[x].teams[i]
                        self.groups[x].scores[i], self.groups[x].scores[j] = self.groups[x].scores[j], self.groups[x].scores[i]
        
        self.knockout_phase[0][0]=self.groups[0].teams[0]
        self.knockout_phase[0][1]=self.groups[1].teams[1]
        
        self.knockout_phase[0][2]=self.groups[2].teams[0]
        self.knockout_phase[0][3]=self.groups[3].teams[1]
        
        self.knockout_phase[0][4]=self.groups[1].teams[0]
        self.knockout_phase[0][5]=self.groups[0].teams[1]
        
        self.knockout_phase[0][6]=self.groups[3].teams[0]
        self.knockout_phase[0][7]=self.groups[2].teams[1]
        
        self.knockout_phase[0][8]=self.groups[4].teams[0]
        self.knockout_phase[0][9]=self.groups[5].teams[1]
        
        self.knockout_phase[0][10]=self.groups[6].teams[0]
        self.knockout_phase[0][11]=self.groups[7].teams[1]
        
        self.knockout_phase[0][12]=self.groups[5].teams[0]
        self.knockout_phase[0][13]=self.groups[4].teams[1]
        
        self.knockout_phase[0][14]=self.groups[7].teams[0]
        self.knockout_phase[0][15]=self.groups[6].teams[1]
        
    def rounds(self): # Do the knockout phase
        for i in range(len(self.knockout_phase)-1):
            for j in range(0,len(self.knockout_phase[i]), 2):
                self.knockout_phase[i+1].append(Match(self.knockout_phase[i][j], self.knockout_phase[i][j+1]).result_calculator(0))
            print("------------------ Phase :" + str(i+1))
            for team in self.knockout_phase[i+1]:
                print(team.to_string())


class Group():
    def __init__(self):
        self.name = None
        self.teams = []
        self.scores = [0, 0, 0, 0]
    
    def add_point(self, team, point=1): # Add a points to a specified team
        for i in range(len(self.teams)):
            if team == self.teams[i]:
                self.scores[i] += point
                return
        
    def match_list(self): # Calculates the scores
        for i in range(len(self.teams)):
            for k in range(len(self.teams) - i - 1):
                winner = Match(self.teams[i], self.teams[i+k +1]).result_calculator()
                if winner is None:
                    self.add_point(self.teams[i])
                    self.add_point(self.teams[i+k +1])
                else:
                    self.add_point(winner, 3)
                    
    def to_string(self):
        tmp = self.name + ":\n"
        for i in range(len(self.teams)):
            tmp += self.teams[i].name + ", points: " + str(self.scores[i]) + " \n"
        return tmp

In [5]:
class DataLoader():
    def __init__(self):
        self.players = []
        self.columns = []
        self.teams = []
        self.groups = []
        
    def load_teams(self, filename, players): # Not working well (useless)
#         from os import listdir
#         from os.path import isfile, join
#         files = [f for f in listdir(path) if isfile(join(path, f))]
        self.columns = []
        first_line = True
        team_name = filename.replace(".csv", "")
        t = Team()
        t.name = team_name
        with open(filename) as f:
            reader = csv.reader(f, delimiter=',')
            for row in reader:
                if first_line:
                    self.columns = row
#                     print("column: " + str(row[self.get_column("Overall")]))
                    first_line = False
                else:
                    player_name = row[self.get_column("Player")]
                    club = row[self.get_column("Club")]
                    t.reserve.append(self.get_player(player_name, players))
                    self.teams.append(t)
        return self.teams
    def get_player(self, name, players, delta = 0.6):
#         print(name)
        name = name.split(' ')[1]
        for player in players:
#             print(player.name)
            player_name = player.name
            if SequenceMatcher(None,player_name,name).ratio() > delta :
                print("Found " + name + " as " + player_name)
                return player
        print("Not found " + name + " as " + player_name)
        
    def load_teams2(self, path, players ): # Loads player in teams
        from os import listdir
        from os.path import isfile, join
        files = [f for f in listdir(path) if isfile(join(path, f))]
        for tmp_file in files:
            tmp_file = tmp_file.replace(".csv","")
            t = Team()
            t.name = tmp_file
            self.teams.append(t)
        for player in players:
            if self.is_player_in_team(player.name,path + player.team + ".csv"):
                self.add_to_team(player,player.team)
        for team in self.teams:
            team.rank = self.get_team_rank(team.name)
            team.quotation = self.get_team_stakes(team.name)
            team.gdp = self.get_team_gdp(team.name)
            team.clean_reserve()
            team.regular_finder()
            team.calculate_average_pace()
            team.power_calculator()
        return self.teams
    
    def get_team_gdp(self, name, filename="../resources/gdp.csv"): # Get the team gdp from file
        self.column = []
        first_line = True
        with open(filename) as f:
            reader = csv.reader(f, delimiter=',')
            for row in reader:
                if first_line:
                    self.columns = row
                    first_line = False
                else:
                    if name == row[self.get_column("Country")]:
                        gdp = row[self.get_column("Gdp")]
                        gdp = gdp.replace('"','')
                        gdp = gdp.replace(",","")
                        return int(gdp)
        
    
    def get_team_rank(self, name, filename="../resources/fifa_ranking.csv"): # Get the team rank from a csv file
        self.column = []
        first_line = True
        with open(filename) as f:
            reader = csv.reader(f, delimiter=',')
            for row in reader:
                if first_line:
                    self.columns = row
                    first_line = False
                else:
                    if name == row[self.get_column("Team")]:
                        return int(row[self.get_column("Rank")])
        rint("Not found rank of: " + name)
                    
    def get_team_stakes(self, name, filename="../resources/stakes.csv"): # Load the team stakes from file
        with open(filename) as f:
            reader = csv.reader(f, delimiter=',')
            for row in reader:
                    if name == row[0]:
                        return float(row[1])
        print("Not found stakes of: " + name)
                    
                
    def load_groups(self,teams, filename): # Loads groups from a csv file
        with open(filename) as f:
            reader = csv.reader(f, delimiter=',')
            for row in reader:
                g = Group()
                g.name = row[0]
                for i in range(1,len(row)):
                    g.teams.append(self.get_team(row[i]))
                self.groups.append(g)
        return self.groups
                
                
                
    def get_team(self, name): # Return the corresponding team
        for team in self.teams:
            if team.name == name:
                return team
        print(name + " is different from " + team.name)
            
    def add_to_team(self, player, team_name): # Insert the player into a team
        for team in self.teams:
            if team.name == team_name:
                team.reserve.append(player)
#         print("Failed to find team "+ team_name)
        
    def is_player_in_team(self, name, filename, delta=0.9):
        self.columns = []
        name = name.split(" ")
        name = name[-1]#(name[1] if len(name) > 1 else name[-1])
        first_line = True
        try:
            with open(filename) as f:
                reader = csv.reader(f, delimiter=',')
                for row in reader:
                    if first_line:
                        self.columns = row
    #                     print("column: " + str(row[self.get_column("Overall")]))
                        first_line = False
                    else:
                        player_name = row[self.get_column("Player")]
                        player_name = player_name.replace("(captain)", "")
                        player_name = player_name.split(" ")
                        if player_name[-1] == " ": 
                            player_name.pop(-1)
                        player_name = player_name[-1]#(name[1] if len(name) > 1 else name[-1])
                        if SequenceMatcher(None,player_name,name).ratio() > delta :
                            return True
        except IOError:
            pass
        return False
        
    def load_players(self, filename):  # Load player dataset from a csv
        self.columns = []
        first_line = True
        with open(filename) as f:
            reader = csv.reader(f, delimiter=',')
            for row in reader:
                if first_line:
                    self.columns = row
#                     print("column: " + str(row[self.get_column("Overall")]))
                    first_line = False
                else:
                    p = Player()
                    p.name = row[self.get_column("Name")]
                    p.age = int(row[self.get_column("Age")])
                    p.stats["overall"] = int(row[self.get_column("Overall")])
                    p.team = row[self.get_column("Nationality")]
                    self.players.append(p)
                    acceleration = int(row[self.get_column("Acceleration")].split("+")[0].split("-")[0])
                    sprint = int(row[self.get_column("Sprint speed")].split("+")[0].split("-")[0])
                    p.stats["pace"] = (acceleration + sprint) / 2
                    '''
                    p.stats["shooting"] = (int(row[self.get_column("Positioning")]) + int(row[self.get_column("Finishing")]) 
                                           + int(row[self.get_column("Long shots")])) / 3
                    p.stats["passing"] = (int(row[self.get_column("Crossing")]) + int(row[self.get_column("Vision")]) +
                                          int(row[self.get_column("Long passing")])) / 3
                    p.stats["dribbling"] = (int(row[self.get_column("Dribbling")]) + 
                                            int(row[self.get_column("Agility")]) +
                                            int(row[self.get_column("Balance")])) / 3
                    p.stats["defending"] = (int(row[self.get_column("Sliding tackle")]) + 
                                            int(row[self.get_column("Standing tackle")]) +
                                            int(row[self.get_column("Marking")])) / 3
                    p.stats["physical"] = (int(row[self.get_column("Strength")]) + 
                                           int(row[self.get_column("Aggression")])) / 2
                    '''
        return self.players

    def get_column(self, title):  # Find the corresponding column
        for i in range(len(self.columns)):
            if self.columns[i] == title:
                return i
        return -1


In [6]:
class Team():
    def __init__(self):
        self.name = None                # Team's name
        self.rank = None             # Team's rank in FIFA
        self.regular = []             # Regular players of the team   ->
        self.reserve = []             # Reserve players of the team   ->
        self.trainer = None             # Team's trainer
        self.quotation = None           # Team's probablity of win
        self.gdp = None  # Team's money. A richer team can grant more services for his players
        self.matchWon = 3               # Matches won by the team
        self.matchLose = 2              # Matches lost by the team
        self.matchDraw = 1              # Matches finished in a draw
        self.regularPower = 0
        self.reservePower = 0
        self.averagePace = 0
        self.power = None

    def regular_power_calculator(self):  # It computes regular players' power points. More powerful are the players, more points will have the team
        Total=0
        for player in self.regular:
            Total+= player.stats.get("overall")
        self.regularPower=Total/len(self.regular)

    def reserve_power_calculator(self):  # It computes reserve players' power points. More powerful are the players, more points will have the team
        Total = 0
        for player in self.reserve:
            Total += player.stats.get("overall")
        self.reservePower=Total/len(self.reserve)

    def power_calculator(self):
        self.regular_power_calculator()
        self.reserve_power_calculator()
        x = 1  # regular moltiplicator
        y = 1  # reserve moltiplicator
        z = 1  # trainer moltiplicator
        a = -1  # quotation moltiplicator
        b = 0.00001  # economicalSituation moltiplicator
        self.power = self.regularPower*x + self.reservePower*y + (self.quotation)*a + self.gdp*b # + self.trainerPower*z + 
        return self.power

    def win_lose_draw_graphics(self):
        import matplotlib.mlab as mlab
        import matplotlib.pyplot as plt
        labels = ['WIN','LOSE','DRAW']
        y = [self.matchWon, self.matchLose, self.matchDraw]
        esplodi = [0, 0, 0] #explode identifica la distanza tra gli elementi
        fig, ax1 = plt.subplots()
        ax1.pie(y, labels=labels, explode=esplodi, autopct='%1.1f%%', startangle=90)
        ax1.set_title(self.name)
        fig.tight_layout()
        fig.show()
    
    def player_ordinator(self): #sort of player based on overoll (selection sort)
        for i in range (0,len(self.reserve)-1):
            index=i
            for j in range(i, len(self.reserve)):
                if (self.reserve[j].stats.get('overall')>self.reserve[index].stats.get('overall')):
                    index=j  
                
            tmp=self.reserve[index]
            self.reserve[index] = self.reserve[i]
            self.reserve[i] = tmp
        
    def regular_finder(self): #found the best eleven player of a team
        self.player_ordinator()
        for i in range (0,11):
            self.regular.append(self.reserve[0])
            self.reserve.pop(0)
                
    def to_string(self): # Returns a string containing all the players
        tmp = ""
        print(self.name + " has rank: "+ str(self.rank) + " , power:" + str(self.power) +" , quotation:" + str(self.quotation) +" gdp:" + str(self.gdp) + " ,AveragePace: " + str(self.averagePace))
#         print("----Regular: " + str(self.regularPower))
#         for player in self.regular:
#             tmp += player.to_string() + "\n"
#         print(tmp)
#         tmp = ""
#         print("----Reserve:" + str(self.reservePower))
#         for player in self.reserve:
#             tmp += player.to_string() + "\n"
        return tmp

    def calculate_average_pace(self):
        summation = 0
        for player in self.regular:
            try:
                summation += player.stats["pace"]
            except:
                print("Failed on:" + player.name)
        for player in self.reserve:
            try:
                summation += player.stats["pace"]
            except:
                print("Failed on:" + player.name)
        self.averagePace = summation / (len(self.regular) + len(self.reserve))
        return self.averagePace
        
    
    def clean_reserve(self, summoned=23): # Remove abundant players and add dummy player if it's missing
        self.reserve = self.reserve[:summoned]
        if len(self.reserve) == 0:
            dummy = Player() # global average values
            dummy.name = "Dummy boy"
            dummy.age = 25
            dummy.stats["overall"] = 65
            dummy.stats["pace"] = 65
            dummy.team = self.name
            self.reserve.append(dummy)
        if len(self.reserve) < summoned:
            average_age = 0
            average_overall = 0
            average_pace = 0
            for player in self.reserve:
                average_age += player.age
                average_overall += player.stats["overall"]
                average_pace += player.stats["pace"]
            average_age /= int(len(self.reserve))
            average_overall /= int(len(self.reserve))
            average_pace /= int(len(self.reserve)) 
            while len(self.reserve) < summoned:
                half_dummy = Player() # Team average values
                half_dummy.name = "Almost a dummy boy"
                half_dummy.age = average_age
                half_dummy.stats["overall"] = average_overall
                half_dummy.team = self.name
                half_dummy.stats["pace"] = average_pace
                self.reserve.append(half_dummy)
   
            
        


In [7]:
class Trainer():
    def __init__(self):
        self.name = None           # Trainer's name
        self.matchesWon = None    # Number if matches won
        self.matchesPlayed = None # Number of matches played
        
    def power_calculator(self, power):
        power=(self.matchesWon/self.matchesPlayed) * 100

In [8]:
'''
def loadFiles():
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    return uploaded
'''

'\ndef loadFiles():\n    from google.colab import files\n    uploaded = files.upload()\n    for fn in uploaded.keys():\n    print(\'User uploaded file "{name}" with length {length} bytes\'.format(\n        name=fn, length=len(uploaded[fn])))\n    return uploaded\n'

In [9]:
def draw_group(group, output_path="../resources/predictions/", input_path="../resources/templates/"):
    from PIL import Image, ImageDraw, ImageFont
    group_name = group.name
    teams = []
    for team in group.teams:
        teams.append(team.name.upper())
    img = Image.open(input_path + "gironi.jpg");
    d = ImageDraw.Draw(img)
    fntSquad = ImageFont.truetype(input_path+'gunship.ttf', 15)
    fntGroups = ImageFont.truetype(input_path+'gunship.ttf', 30)

    x=30
    for i in range (0,4):
        d.text((x,293), teams[i],  font=fntSquad, fill=(255, 255, 255)) 
        x+=146
    d.text((202,96), group_name,  font=fntGroups, fill=(255, 255, 255))
    img.save(output_path+ group_name + '.jpg')
    img = Image.open(output_path+ group_name + '.jpg')
    for i in range(len(teams)):
#     i = 0
        img2 = Image.open(input_path+"flags/" +teams[i].lower().capitalize().replace(" ","-")+ ".png")
        width, height = img2.size
        area = (9, 12, width-2,height-12) 
        img2= img2.crop(area)
        img2= img2.resize((int(width*1.75),int(height*1.1)))
        img.paste(img2, (35 * (i+1) + i * (int(width*1.75)),196))
    img.save(output_path+ group_name + '.jpg')
    

In [10]:
def main():
    import pickle
    # #     Load data from csv
    loader = DataLoader()
    players = loader.load_players("../resources/CompleteDataset.csv")
    #     print(players[0])
    teams = loader.load_teams2("../resources/nations/", players)
    print(len(teams))
    for team in teams:
        print(team.to_string())
    pickle.dump( teams, open( "../resources/saved_teams.p", "wb" ) )
    groups = loader.load_groups(teams,"../resources/groups.csv")
    pickle.dump( groups, open( "../resources/saved_groups.p", "wb" ) )

    # # Load data from pickle
    #     teams = pickle.load( open( "../resources/saved_teams.p", "rb" ) )
    #     groups = pickle.load( open( "../resources/saved_groups.p", "rb" ) )

    w = WorldCup("2018", groups)
    w.calculate_match_list()
    w.pre_rounds()
    
    for group in w.groups:
        draw_group(group)

    for group in w.groups:
        print(group.to_string())
    print("----------------Phase 0")
    for e in w.knockout_phase[0]:
        print(e.to_string())
    w.rounds()
    
#     print("----------------")
#     for e in w.knockout_phase[0]:
#         print(e.to_string())

In [11]:
main()

32
Serbia has rank: 37 , power:50.41432 , quotation:100.0 gdp:41432 ,AveragePace: 64

Japan has rank: 55 , power:-155.27863 , quotation:350.0 gdp:4872137 ,AveragePace: 64

Brazil has rank: 2 , power:179.30506 , quotation:5.25 gdp:2055506 ,AveragePace: 73

Sweden has rank: 18 , power:-44.6196 , quotation:200.0 gdp:538040 ,AveragePace: 59

Germany has rank: 1 , power:193.77439 , quotation:9.0 gdp:3677439 ,AveragePace: 69

Senegal has rank: 23 , power:25.16375 , quotation:125.0 gdp:16375 ,AveragePace: 69

South Korea has rank: 59 , power:-604.69249 , quotation:750.0 gdp:1530751 ,AveragePace: 65

Morocco has rank: 40 , power:-51.90861 , quotation:200.0 gdp:109139 ,AveragePace: 68

Egypt has rank: 31 , power:43.35369 , quotation:100.0 gdp:235369 ,AveragePace: 69

Australia has rank: 39 , power:-848.76579 , quotation:1000.0 gdp:1323421 ,AveragePace: 70

Poland has rank: 7 , power:-46.7549 , quotation:200.0 gdp:524510 ,AveragePace: 67

Switzerland has rank: 8 , power:57.78887 , quotation:100.